In [555]:
import pandas as pd
import imageio 
import re
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from sklearn.utils import shuffle
from skimage import color



np.set_printoptions(threshold=np.inf)
from matplotlib import pyplot as plt

class primusSequence:

    def __init__(self, folder, package):
        self.image = imageio.imread("./data/" + package + "/" + folder + "/" + folder + ".png")
        self.semantic = open("./data/" + package + "/" + folder + "/" + folder + ".semantic").read()
        self.labels = re.findall(r'note\-(.+?)\_', self.semantic)
        
    def flattened_image(self):
        im = self.image
        return(im.flatten())
                                    
        
def init_objs(dataset):
    
    if dataset == 'train':
        ROOT = './data/package_aa/'
        package = 'package_aa'
    elif dataset == 'validate':
        ROOT = './data/package_ab/'
        package = 'package_ab'

    folders = [directory for directory in os.listdir(ROOT) if os.path.isdir(ROOT+directory)]
    objs = []
    
    for f in folders:
        objs.append(primusSequence(f, package))
        
    return(objs)

In [76]:
objs = init_objs('train')

In [146]:
train_labels = ['a', 'b'] * 21797

In [148]:
train_images = np.array(train_images)

/Users/George/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [612]:
def map_pitch(pitch):
    
    m = {"a": 0,
         "h": 1,
         "c1": 2,
         "d1": 3,
         "e1": 4,
         "f1": 5,
         "g1": 6,
         "a1": 0,
         "h1": 1,
         "c2": 2,
         "d2": 3,
         "e2": 4,
         "f2": 5,
         "g2": 6,
         "a2": 0,
         "h2": 1,
         "c3": 2,
         "other": 7}
    
    return(m[pitch])

class ViennaNote:
    
    def __init__(self, fn):
        attr = fn.split('-')
        self.type = attr[0]
        self.time = attr[1]
        self.pitch = attr[2]
        imfile = imageio.imread('./data/originals-resized/' + fn)
        self.image = color.rgb2gray(imfile) / 255
        
    def resize_image(self, width, height):
        pass
    
class InputData:
    
    ROOT = './data/originals-resized/' 
    
    def __init__(self, path = ROOT):
        self.files = [file for file in os.listdir(path) if os.path.isfile(path + file) and file[0:4] == 'note']
        self.objs = [ViennaNote(f) for f in self.files]
                
    def training_images(self, path = ROOT):
        images = [o.image for o in self.objs]
        return(np.array(images))
    
    def training_labels(self, path = ROOT):
        notes = [o.pitch for o in self.objs]
        labels = [map_pitch(pitch) for pitch in notes]
        return(np.array(labels))

In [613]:
input_data = InputData()

In [614]:
x = input_data.training_images()

In [442]:
x = np.array(x)

In [552]:
import cv2

In [615]:
train_images = input_data.training_images()
train_labels = input_data.training_labels()

In [643]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (50, 30)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(8, activation=tf.nn.softmax),
])

def ctc_loss(y_true, y_pred, input_length, label_length, real_y_true_ts):
    return tf.keras.backend.ctc_batch_cost(real_y_true_ts, y_pred, input_length, label_length)

In [644]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [646]:
train_images, train_labels = shuffle(train_images, train_labels)

model.fit(train_images, 
          train_labels,
          validation_split=0.1,
          epochs=100)

Epoch 1/100
92/92 [==============================] - 0s 3ms/step - loss: 0.7540 - accuracy: 0.7534 - val_loss: 0.7310 - val_accuracy: 0.7622
Epoch 2/100
92/92 [==============================] - 0s 3ms/step - loss: 0.7573 - accuracy: 0.7385 - val_loss: 0.8116 - val_accuracy: 0.6951
Epoch 3/100
92/92 [==============================] - 0s 3ms/step - loss: 0.7124 - accuracy: 0.7707 - val_loss: 0.8053 - val_accuracy: 0.7317
Epoch 4/100
92/92 [==============================] - 0s 4ms/step - loss: 0.7000 - accuracy: 0.7758 - val_loss: 0.8606 - val_accuracy: 0.7195
Epoch 5/100
92/92 [==============================] - 0s 4ms/step - loss: 0.6751 - accuracy: 0.7809 - val_loss: 0.8046 - val_accuracy: 0.7378
Epoch 6/100
92/92 [==============================] - 0s 4ms/step - loss: 0.6621 - accuracy: 0.7846 - val_loss: 0.7792 - val_accuracy: 0.7470
Epoch 7/100
92/92 [==============================] - 0s 5ms/step - loss: 0.6729 - accuracy: 0.7721 - val_loss: 0.8141 - val_accuracy: 0.7256
Epoch 8/100
9

Epoch 59/100
92/92 [==============================] - 0s 3ms/step - loss: 0.2091 - accuracy: 0.9310 - val_loss: 1.0503 - val_accuracy: 0.7409
Epoch 60/100
92/92 [==============================] - 0s 3ms/step - loss: 0.1719 - accuracy: 0.9531 - val_loss: 0.9134 - val_accuracy: 0.7683
Epoch 61/100
92/92 [==============================] - 0s 3ms/step - loss: 0.1984 - accuracy: 0.9375 - val_loss: 0.9641 - val_accuracy: 0.7378
Epoch 62/100
92/92 [==============================] - 0s 3ms/step - loss: 0.1884 - accuracy: 0.9440 - val_loss: 0.9920 - val_accuracy: 0.7500
Epoch 63/100
92/92 [==============================] - 0s 3ms/step - loss: 0.1456 - accuracy: 0.9626 - val_loss: 0.9520 - val_accuracy: 0.7774
Epoch 64/100
92/92 [==============================] - 0s 3ms/step - loss: 0.2028 - accuracy: 0.9280 - val_loss: 1.1522 - val_accuracy: 0.7073
Epoch 65/100
92/92 [==============================] - 0s 3ms/step - loss: 0.1835 - accuracy: 0.9399 - val_loss: 1.1336 - val_accuracy: 0.7439
Epoch 

In [ ]:
model.summary()

In [320]:
shuff

In [597]:
train_labels.shape

(3272,)